## Data Analysis of Head Start Accessibility and Impact across the Continental U.S.

- Presented by team <b>Data Miners</b> for <b>2023 AGA Datathon</b>

In [411]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
import math
import sys
import geopy.distance
from itertools import combinations
from sklearn.linear_model import LinearRegression

pd.options.display.float_format = '{:.5f}'.format

#### Loading Datasets

##### List of data source:
- Head Start Program Facts: Fiscal Year 2021: https://eclkc.ohs.acf.hhs.gov/about-us/article/head-start-program-facts-fiscal-year-2021

- Head Start Center Locations: https://eclkc.ohs.acf.hhs.gov/sites/default/files/locatordata/ALL_all.csv

- SAIPE Child Poverty Data: https://www.census.gov/data/datasets/2021/demo/saipe/2021-state-and-county.html

- U.S. States Economic Data: https://apps.bea.gov/regional/downloadzip.cfm

In [412]:
STATE_FILTER = ['US', 'PR', 'VI', 'HI', 'GU', 'MP', 'AS', 'AK']

In [413]:
df_state_abbr = pd.read_csv('states-abbr.csv')
df_state_abbr.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [414]:
def load_transform_headstart_fiscal() -> pd.DataFrame:
    df = pd.read_html('https://eclkc.ohs.acf.hhs.gov/about-us/article/head-start-program-facts-fiscal-year-2021')
    df =  df[1].iloc[2:, :3]
    df.columns = ['state_name', 'federal_funding', 'enrollment_count', ]
    df = pd.merge(df, df_state_abbr, left_on='state_name', right_on='State', how='left').drop('State', axis=1)
    df.columns = ['state_name', 'federal_funding', 'enrollment_count', 'state' ]
    df.federal_funding = df.federal_funding.map(lambda s: s.replace('$', '').replace(',', '')).astype(float)
    df.enrollment_count = df.enrollment_count.astype(int)
    df = df[df.state.map(lambda s: s not in STATE_FILTER)]
    return df

df_headstart_fiscal = load_transform_headstart_fiscal()
print(df_headstart_fiscal.shape)
df_headstart_fiscal.head()

(57, 4)


,state_name,federal_funding,enrollment_count,state
0,Alabama,156825889.00000,13438,AL
2,Arizona,175213573.00000,13054,AZ
3,Arkansas,106372981.00000,9191,AR
4,California,1320500737.00000,87819,CA
5,Colorado,119296359.00000,9489,CO


In [415]:
def read_state_econ() -> pd.DataFrame:
    tar_dir = './SASUMMARY'
    file_state_pairs = os.listdir(tar_dir)

    file_state_pairs = [(file, file[10:12]) for file in file_state_pairs if os.path.isfile(os.path.join(tar_dir, file)) and file.__contains__('1998_2022') and not file.__contains__('ALL_AREAS')]

    df = pd.DataFrame(columns=['state', 'rgdp', 'personal_income'])

    for file, state in file_state_pairs:
        df_temp = pd.read_csv(os.path.join(tar_dir, file))
        rgdp = df_temp.iloc[0, 32]
        personal_income = df_temp.iloc[4, 32]
        df.loc[len(df)] = [state, rgdp, personal_income]

    df = df[df.state.map(lambda s: s not in STATE_FILTER)]
    return df


df_state_economic = read_state_econ()
df = pd.merge(df_headstart_fiscal, df_state_economic, on='state', how='right')
df.personal_income = df.personal_income.astype(float)
df.rgdp = df.rgdp.astype(float)
df.head()

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income
0,Alabama,156825889.00000,13438,AL,235807.30000,258362.20000
1,Arkansas,106372981.00000,9191,AR,137355.50000,160254.20000
2,Arizona,175213573.00000,13054,AZ,403474.20000,430083.50000
3,California,1320500737.00000,87819,CA,3167460.80000,3006647.30000
4,Colorado,119296359.00000,9489,CO,416114.40000,442213.00000


In [416]:
df.corr()

,federal_funding,enrollment_count,rgdp,personal_income
federal_funding,1.00000,0.98014,0.96825,0.96529
enrollment_count,0.98014,1.00000,0.94490,0.95068
rgdp,0.96825,0.94490,1.00000,0.99457
personal_income,0.96529,0.95068,0.99457,1.00000


<AxesSubplot:>

In [417]:
sns.heatmap(df.corr(), annot=True,)

##### <b>Insight</b>
- For effective funding, each state should receive funding in proportion to its personal income and the number of children in need of Head Start. The table demonstrates strong positive correlation between federal funding, local personal income, and local enrollment count, indicating a successful funding policy is enforced across the states.

#### <b>Metric: Fund per child</b>
- Definition:  The total amount of head start program fund divided by the number of children enrolled in the head start centers within a state.
- Usage: 

- - Shows the distribution of fiscal resource across the states. 
- - Identify states that are overfunded and underfunded.

In [418]:
df['fund_per_child'] = df.federal_funding / df.enrollment_count.astype(float)
df.sort_values('fund_per_child', ascending=True).head()

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income,fund_per_child
35,Oregon,105339943.00000,12899,OR,254708.40000,264173.80000,8166.52012
47,West Virginia,75200678.00000,7824,WV,76525.50000,88744.90000,9611.53860
36,Pennsylvania,355878475.00000,35372,PA,772335.90000,836778.30000,10061.02214
16,Louisiana,209932116.00000,20165,LA,231262.30000,250170.60000,10410.71738
23,Mississippi,225587275.00000,21622,MS,114152.90000,136330.20000,10433.22889


#### <b>Metric: Funding Index</b>
- Definition: The amount of Head Start funding divided by local personal income, then devided by the number of enrolled children.
- Usage: 
- - Offset the impact of local price level.
- - Measure the intensity of funding as shared by each child. (Measuring how strong the funding is when applied to an individual child)

In [419]:
df['funding_index'] = df.federal_funding.astype(float) / df.personal_income.astype(float) / df.enrollment_count.astype(float)
df.sort_values('funding_index', ascending=True).head(10)

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income,fund_per_child,funding_index
3,California,1320500737.00000,87819,CA,3167460.80000,3006647.30000,15036.61778,0.00500
41,Texas,719676508.00000,68467,TX,1924007.50000,1879419.60000,10511.29023,0.00559
8,Florida,442454135.00000,40319,FL,1218430.20000,1441598.90000,10973.83702,0.00761
32,New York,626590038.00000,46051,NY,1763524.60000,1483802.30000,13606.43717,0.00917
36,Pennsylvania,355878475.00000,35372,PA,772335.90000,836778.30000,10061.02214,0.01202
12,Illinois,410844005.00000,29117,IL,864170.90000,851243.10000,14110.10767,0.01658
33,Ohio,383581783.00000,33303,OH,689681.30000,679233.00000,11517.93481,0.01696
29,New Jersey,193901043.00000,15267,NJ,646731.30000,714990.20000,12700.66437,0.01776
9,Georgia,258763401.00000,23259,GA,655827.30000,617553.00000,11125.30208,0.01802
43,Virginia,151962333.00000,13886,VA,576964.30000,599042.00000,10943.56424,0.01827


In [420]:
df.funding_index.describe()

count   49.00000
mean     0.07927
std      0.08925
min      0.00500
25%      0.02371
50%      0.04161
75%      0.10831
max      0.42107
Name: funding_index, dtype: float64

<AxesSubplot:>

In [421]:
df.funding_index.plot.box()

##### <b>Insight</b>
- Although the funding policy has taken into account regional economic status and enrollment, the child funding index shows that there exists inequality regarding the funding policy. In some states, the effect of funding proves stronger.
- Policy makers may review their existing funding strategy and pay more attention to the states where the funding index is low.

In [422]:
df_headstart_center = pd.read_csv('ALL_all.csv',)
df_headstart_center.head()

,addressLineOne,addressLineTwo,cid,grantNumber,delegateNumber,type,longitude,latitude,name,city,...,programAddressLineOne,programAddressLineTwo,programName,programCity,programState,programZipFive,programZipFour,programPhone,programRegistrationPhone,programIdentifier
0,11500 Joslin St,NaN,100000,09CH011157,2,1,-118.08870,33.94430,Lakeview (Plaza),Santa Fe Springs,...,13300 Crossroads Pkwy N,Ste 440,Plaza de la Raza Child Development Services Inc.,City Of Industry,CA,91746.00000,3440,(562) 929-6320,(562)776-1301,los_angeles_county_office_of_education_ca
1,5036 North Huntington Drive,NaN,100001,09CH011157,2,1,-118.17160,34.08910,El Sereno (Plaza),Los Angeles,...,13300 Crossroads Pkwy N,Ste 440,Plaza de la Raza Child Development Services Inc.,City Of Industry,CA,91746.00000,3440,(323) 222-2786,(562)776-1301,los_angeles_county_office_of_education_ca
2,2141 Workman St,NaN,100002,09CH011157,2,1,-118.21440,34.07060,Cedillo (Plaza),Los Angeles,...,13300 Crossroads Pkwy N,Ste 440,Plaza de la Raza Child Development Services Inc.,City Of Industry,CA,91746.00000,3440,(323) 225-9792,(562)776-1301,los_angeles_county_office_of_education_ca
3,12322 Pellissier Rd,NaN,100003,09CH011157,2,1,-118.03220,34.03370,Little Star (Plaza),Whittier,...,13300 Crossroads Pkwy N,Ste 440,Plaza de la Raza Child Development Services Inc.,City Of Industry,CA,91746.00000,3440,(562) 699-5655,(562)776-1301,los_angeles_county_office_of_education_ca
4,12830 Columbia Way,NaN,100004,09CH011157,2,1,-118.13410,33.91410,Columbia (Plaza),Downey,...,13300 Crossroads Pkwy N,Ste 440,Plaza de la Raza Child Development Services Inc.,City Of Industry,CA,91746.00000,3440,(562) 803-6229,(562)776-1301,los_angeles_county_office_of_education_ca


In [423]:
df_headstart_center.columns

Index(['addressLineOne', 'addressLineTwo', 'cid', 'grantNumber',
       'delegateNumber', 'type', 'longitude', 'latitude', 'name', 'city',
       'state', 'zipFive', 'zipFour', 'phone', 'googleMapZoomLevel', 'county',
       'programCounty', 'programAddressLineOne', 'programAddressLineTwo',
       'programName', 'programCity', 'programState', 'programZipFive',
       'programZipFour', 'programPhone', 'programRegistrationPhone',
       'programIdentifier'],
      dtype='object')

In [424]:
# to perform county level analysis, we need to combine state and county so as to make it unique
df_headstart_center['state_county'] = df_headstart_center['state'] + " " + df_headstart_center['county']
df_headstart_center.shape, df_headstart_center.cid.nunique(), df_headstart_center.state_county.nunique()

((21334, 28), 21334, 2895)

In [425]:
df_saipe = pd.read_excel('est21all.xls',).drop(index=[0, 1])
state_names = df_state_abbr.State.tolist()
filter_out = state_names + ['United States']
df_saipe = df_saipe[~df_saipe['Unnamed: 3'].isin(filter_out)]   # fetch only county level data, filtering out state level data

df_saipe.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
5,01,001,AL,Autauga County,6296,4772,7820,10.70000,8.10000,13.30000,...,20.40000,66444,60061,72827,.,.,.,.,.,.
6,01,003,AL,Baldwin County,25526,21599,29453,10.80000,9.10000,12.50000,...,18.50000,65658,60723,70593,.,.,.,.,.,.
7,01,005,AL,Barbour County,5089,3773,6405,23,17,29,...,44.60000,38649,34308,42990,.,.,.,.,.,.
8,01,007,AL,Bibb County,4204,3324,5084,20.60000,16.30000,24.90000,...,35.40000,48454,42438,54470,.,.,.,.,.,.


In [426]:
def to_int(v):
    try:
        return int(v)
    except Exception as e:
        print(e)
        print(v)
        return 0
    
teen_poverty = df_saipe.iloc[1:, 10].map(to_int)
young_poverty = df_saipe.iloc[1:, 16].map(to_int)
child_poverty = teen_poverty - young_poverty

invalid literal for int() with base 10: '.'
.
invalid literal for int() with base 10: '.'
.


In [427]:
df_child_poverty = pd.DataFrame({
    'state': df_saipe.iloc[1:, 2],
    'county': df_saipe.iloc[1:, 3],
    'county_fips': df_saipe.iloc[1:, 0],
    'child_poverty_count': child_poverty.values,    # estimated number of children in poverty
})
df_child_poverty = df_child_poverty[~df_child_poverty.state.isin(STATE_FILTER)]
df_child_poverty = df_child_poverty[df_child_poverty.county.str.contains('County|Parish')] # only study county level data
df_child_poverty = pd.merge(df_child_poverty, df_state_abbr, left_on='state', right_on='Abbreviation', how='left',).drop(columns=['Abbreviation'])
col = df_child_poverty.columns.tolist()
col[-1] = 'state_name'
df_child_poverty.columns = col
df_child_poverty['state_county'] = df_child_poverty.state + ' ' + df_child_poverty.county
df_child_poverty.head()

,state,county,county_fips,child_poverty_count,state_name,state_county
0,AL,Autauga County,01,592,Alabama,AL Autauga County
1,AL,Baldwin County,01,2445,Alabama,AL Baldwin County
2,AL,Barbour County,01,500,Alabama,AL Barbour County
3,AL,Bibb County,01,335,Alabama,AL Bibb County
4,AL,Blount County,01,637,Alabama,AL Blount County


In [428]:
print(df_child_poverty.shape, df_child_poverty.state_county.nunique())
df_child_poverty.head()

(3066, 6) 3066


,state,county,county_fips,child_poverty_count,state_name,state_county
0,AL,Autauga County,01,592,Alabama,AL Autauga County
1,AL,Baldwin County,01,2445,Alabama,AL Baldwin County
2,AL,Barbour County,01,500,Alabama,AL Barbour County
3,AL,Bibb County,01,335,Alabama,AL Bibb County
4,AL,Blount County,01,637,Alabama,AL Blount County


##### <b>Metric: Children per Center (CPC)</b>
- Definition: The number of enrolled children divided by the number of centers. Can be calculated at either county or state level. In this analysis it is calculated at county level.
- Usage: 

- - Evaluate the average workload of head start centers within a region. If the metric goes too high, the children in the corresponding area may be underserved because the resources are shared by too many children. If the metric goes too low, the centers in the corresponding area may not be fully utilized.

In [429]:
df.head()

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income,fund_per_child,funding_index
0,Alabama,156825889.00000,13438,AL,235807.30000,258362.20000,11670.32959,0.04517
1,Arkansas,106372981.00000,9191,AR,137355.50000,160254.20000,11573.60255,0.07222
2,Arizona,175213573.00000,13054,AZ,403474.20000,430083.50000,13422.21334,0.03121
3,California,1320500737.00000,87819,CA,3167460.80000,3006647.30000,15036.61778,0.00500
4,Colorado,119296359.00000,9489,CO,416114.40000,442213.00000,12572.06861,0.02843


In [430]:
def get_child_per_center(children:float, centers: float):
    if centers == 0 or centers == np.nan:
        return np.nan
    return round(children / centers, 5)


def find_county_cpc(index):
    state_county = df_child_poverty.loc[index, 'state_county']
    count_children = df_child_poverty.loc[index, 'child_poverty_count']
    count_centers = df_headstart_center[df_headstart_center.state_county == state_county].shape[0]
    return get_child_per_center(count_children, count_centers)


def find_state_cpc(index):
    state = df_child_poverty.loc[index, 'state']
    count_enrolled_children = df[df.state == state].enrollment_count.values[0]
    count_centers = df_headstart_center[df_headstart_center.state == state].shape[0]
    return get_child_per_center(count_enrolled_children, count_centers)


def check_has_center(index):
    state_county = df_child_poverty.loc[index, 'state_county']
    count_centers = df_headstart_center[df_headstart_center.state_county == state_county].shape[0]
    return count_centers > 0


df_child_poverty['cpc'] = df_child_poverty.index.map(find_county_cpc)
df_child_poverty['has_center'] = df_child_poverty.index.map(check_has_center)
df_child_poverty['state_cpc'] = df_child_poverty.index.map(find_state_cpc)
df_child_poverty.head()

,state,county,county_fips,child_poverty_count,state_name,state_county,cpc,has_center,state_cpc
0,AL,Autauga County,01,592,Alabama,AL Autauga County,296.00000,True,42.12539
1,AL,Baldwin County,01,2445,Alabama,AL Baldwin County,489.00000,True,42.12539
2,AL,Barbour County,01,500,Alabama,AL Barbour County,500.00000,True,42.12539
3,AL,Bibb County,01,335,Alabama,AL Bibb County,335.00000,True,42.12539
4,AL,Blount County,01,637,Alabama,AL Blount County,127.40000,True,42.12539


In [431]:
df_child_poverty.shape[0], df_child_poverty.county.nunique() # counties may have same name but different state

(3066, 1801)

In [432]:
df_county_boundaries = pd.read_html('https://en.wikipedia.org/wiki/User:Michael_J/County_table')[0].iloc[:, 1:]

In [433]:
def to_float(s):
    try:
        return float(s.replace('–', '-'))
    except Exception as e:
        print(e)
        print(s)
        return 0


col = df_county_boundaries.columns.map(lambda s: s.lower()).tolist()
col[2] = 'county'
df_county_boundaries.columns = col
df_county_boundaries.latitude = df_county_boundaries.latitude.map(lambda s: to_float(s[:-1]))
df_county_boundaries.longitude = df_county_boundaries.longitude.map(lambda s: to_float(s[:-1]))
df_county_boundaries.head()

,state,fips,county,county seat(s) [3],population(2010),land areakm²,land areami²,water areakm²,water areami²,total areakm²,total areami²,latitude,longitude
0,AL,1001,Autauga,Prattville,54571,1539.58200,594.43600,25.776,9.952,1565.35800,604.38800,32.53638,-86.64449
1,AL,1003,Baldwin,Bay Minette,182265,4117.52200,1589.78400,1133.190,437.527,5250.71200,2027.31100,30.65922,-87.74607
2,AL,1005,Barbour,Clayton,27457,2291.81900,884.87600,50.865,19.639,2342.68400,904.51500,31.87067,-85.40546
3,AL,1007,Bibb,Centreville,22915,1612.48100,622.58200,9.289,3.587,1621.77000,626.16900,33.01589,-87.12715
4,AL,1009,Blount,Oneonta,57322,1669.96200,644.77600,15.157,5.852,1685.11900,650.62800,33.97745,-86.56725


In [434]:
df_county_boundaries['state_county'] = df_county_boundaries.state + ' ' + df_county_boundaries.county + " County"
df_child_poverty = pd.merge(df_child_poverty, df_county_boundaries[['state_county', 'latitude', 'longitude']], on='state_county', how='left')
df_child_poverty.head()

,state,county,county_fips,child_poverty_count,state_name,state_county,cpc,has_center,state_cpc,latitude,longitude
0,AL,Autauga County,01,592,Alabama,AL Autauga County,296.00000,True,42.12539,32.53638,-86.64449
1,AL,Baldwin County,01,2445,Alabama,AL Baldwin County,489.00000,True,42.12539,30.65922,-87.74607
2,AL,Barbour County,01,500,Alabama,AL Barbour County,500.00000,True,42.12539,31.87067,-85.40546
3,AL,Bibb County,01,335,Alabama,AL Bibb County,335.00000,True,42.12539,33.01589,-87.12715
4,AL,Blount County,01,637,Alabama,AL Blount County,127.40000,True,42.12539,33.97745,-86.56725


In [435]:
no_pos_state_county = df_child_poverty[df_child_poverty.latitude.isna() | df_child_poverty.longitude.isna()]['state_county']
df_avg_pos = df_headstart_center[df_headstart_center.state_county.isin(no_pos_state_county)].groupby('state_county').agg({'latitude': 'mean', 'longitude': 'mean'})


def find_lat(index):
    state_county = df_child_poverty.loc[index, 'state_county']
    if state_county not in df_avg_pos.index:
        return np.nan
    lat = df_avg_pos.loc[state_county, 'latitude']
    return lat

def find_long(index):
    state_county = df_child_poverty.loc[index, 'state_county']
    if state_county not in df_avg_pos.index:
        return np.nan
    long = df_avg_pos.loc[state_county, 'longitude']
    return long

df_child_poverty.loc[df_child_poverty.latitude.isna(), 'latitude'] = df_child_poverty[df_child_poverty.latitude.isna()].index.map(find_lat)
df_child_poverty.loc[df_child_poverty.longitude.isna(), 'longitude'] = df_child_poverty[df_child_poverty.longitude.isna()].index.map(find_long)
df_child_poverty.latitude.isna().sum(), df_child_poverty.longitude.isna().sum()

(7, 7)

In [436]:
df_child_poverty = df_child_poverty[~(df_child_poverty.latitude.isna() | df_child_poverty.longitude.isna()) & df_child_poverty.state_county.str.contains('County|Parish')]
df_child_poverty.shape

(3059, 11)

In [437]:
df_child_poverty.latitude.isna().sum(), df_child_poverty.longitude.isna().sum()

(0, 0)

##### <b>Metric: Center Coverage Rate</b>
- Definition: The number of regions where head start is available divided by the number of regions. The regions can be either county, state, or other well-defined unit. In our analysis it is calculated at county level.
- Usage: 
- - Gives an overall idea of the accessibility of head start program across the nation.

In [438]:
df_child_poverty.has_center.mean()  # county level center coverage rate

0.897025171624714

##### <b>Metric: Enrollment Rate</b>
- Definition: The number of children enrolled in the head start centers of a county divided by the estimated number of children in poverty in the same county. (Can also be calculated at other geospatial scope such as state level)
- Usage: 
- - Shows how many community members are actually participating in the head start program in a county.

In [439]:
df.head()

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income,fund_per_child,funding_index
0,Alabama,156825889.00000,13438,AL,235807.30000,258362.20000,11670.32959,0.04517
1,Arkansas,106372981.00000,9191,AR,137355.50000,160254.20000,11573.60255,0.07222
2,Arizona,175213573.00000,13054,AZ,403474.20000,430083.50000,13422.21334,0.03121
3,California,1320500737.00000,87819,CA,3167460.80000,3006647.30000,15036.61778,0.00500
4,Colorado,119296359.00000,9489,CO,416114.40000,442213.00000,12572.06861,0.02843


In [440]:
df_state_child_poverty = df_child_poverty.groupby('state')['child_poverty_count'].sum().to_frame()
df_state_child_poverty.reset_index(inplace=True)
df_state_child_poverty = pd.merge(df_state_child_poverty, df_state_abbr, left_on='state',  right_on='Abbreviation', how='left').drop(columns=['Abbreviation'])
col = df_state_child_poverty.columns.tolist()
col[-1] = 'state_name'
df_state_child_poverty.columns = col
df_state_child_poverty = pd.merge(
    df_state_child_poverty, df, on='state', how='left', suffixes=('', '_headstart')).drop(columns=['state_name_headstart']
    )
df_state_child_poverty['enroll_rate'] = df_state_child_poverty.enrollment_count.astype(float) / df_state_child_poverty.child_poverty_count.astype(float)
df_state_child_poverty.sort_values('enroll_rate', ascending=True).head()

,state,child_poverty_count,state_name,federal_funding,enrollment_count,rgdp,personal_income,fund_per_child,funding_index,enroll_rate
30,NV,38827,Nevada,41621944.00000,2937,187226.20000,197290.90000,14171.58461,0.07183,0.07564
10,ID,19157,Idaho,38883862.00000,2908,91683.70000,109775.90000,13371.34182,0.12181,0.15180
2,AZ,83855,Arizona,175213573.00000,13054,403474.20000,430083.50000,13422.21334,0.03121,0.15567
8,GA,143901,Georgia,258763401.00000,23259,655827.30000,617553.00000,11125.30208,0.01802,0.16163
40,TX,414914,Texas,719676508.00000,68467,1924007.50000,1879419.60000,10511.29023,0.00559,0.16501


In [441]:
df_child_poverty.to_csv('child_poverty_merged_10-22.csv', index=False)
df_state_child_poverty.to_csv('enroll_and_fund.csv', index=False)

##### <b>Distribution of Children per Center</b>
- Skewed to the right, meaning that there are few counties with abnormaly high figures of children per center. 

<AxesSubplot:>

In [442]:
sns.boxplot(df_child_poverty['cpc'],)

In [443]:
df_child_poverty.cpc.describe()

count   2744.00000
mean     174.76630
std      202.66652
min        5.66667
25%       67.35268
50%      120.34167
75%      211.90625
max     4635.00000
Name: cpc, dtype: float64

In [444]:
df_child_poverty[['cpc','state', 'state_county']].sort_values(ascending=False, by='cpc' ).head(10)

,cpc,state,state_county
2545,4635.00000,TX,TX Denton County
2607,2693.00000,TX,TX Jefferson County
2527,2543.00000,TX,TX Collin County
389,2078.66667,GA,GA Cobb County
1916,1857.00000,NC,NC New Hanover County
423,1406.42857,GA,GA Gwinnett County
1179,1383.00000,MD,MD Wicomico County
425,1333.00000,GA,GA Hall County
431,1331.50000,GA,GA Henry County
2654,1287.00000,TX,TX Montgomery County


##### <b>Insight and Policy Suggestion</b>
- As shown, there are many Head Start centers in Texas and Georgia that have abnormaly high child per center values.
- Agencies and program managers may need to focus on those counties and states and try to provide more resource so as to serve the communities there.

##### <b>Distribution of Funding Index</b>
- Skewed to the right, meaning that there are few states where the intensity of funding is significantly higher than all the other states.

<AxesSubplot:>

In [445]:
df.funding_index.plot.box()

In [446]:
df.funding_index.describe()

count   49.00000
mean     0.07927
std      0.08925
min      0.00500
25%      0.02371
50%      0.04161
75%      0.10831
max      0.42107
Name: funding_index, dtype: float64

##### <b>Insight and Policy Suggestion</b>
- As shown, there are many Head Start centers in Texas and Georgia that have abnormaly high child per center values.
- To better facilitate equity and accessibility, policy makers may consider to rebalance the current funding policy to reduce the difference of funding index across the states.

#### <b>An Econometric Model for the Relationship between Head Start Funding, Enrollment, and Income Level

In [447]:
md = LinearRegression()
X = df[['enrollment_count']]
y = df.federal_funding
md.fit(X, y)
r2 = md.score(X, y)

n = len(y)
p = X.shape[1]
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print(f'r2: {r2}, r2_adj: {r2_adj}')
print(md.coef_, md.intercept_)

r2: 0.9606775994029817, r2_adj: 0.9598409525817685
[12972.55153715] -12203066.093651742


In [448]:
md = LinearRegression()
X = df[['enrollment_count', 'personal_income']]
y = df.federal_funding
md.fit(X, y)
r2 = md.score(X, y)

n = len(y)
p = X.shape[1]
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print(f'r2: {r2}, r2_adj: {r2_adj}')
print(md.coef_, md.intercept_)

r2: 0.9723373147180877, r2_adj: 0.971134589271048
[8592.14348213  144.92535648] -8901418.68396768


- Enrollment alone can determine to a very large extent the funding received by a state.
- Personal income can further improve the model's R2 score, showing significant impact on the funding. The positive coefficient of income also suggests that funding granted to each state were adjusted slightly based on local price level.

In [449]:
df.head()

,state_name,federal_funding,enrollment_count,state,rgdp,personal_income,fund_per_child,funding_index
0,Alabama,156825889.00000,13438,AL,235807.30000,258362.20000,11670.32959,0.04517
1,Arkansas,106372981.00000,9191,AR,137355.50000,160254.20000,11573.60255,0.07222
2,Arizona,175213573.00000,13054,AZ,403474.20000,430083.50000,13422.21334,0.03121
3,California,1320500737.00000,87819,CA,3167460.80000,3006647.30000,15036.61778,0.00500
4,Colorado,119296359.00000,9489,CO,416114.40000,442213.00000,12572.06861,0.02843
